In [3]:
# This tutorial is meant as an introduction to some of BrainEx's important parameters. Knowing these parameters
# will allow you greater control over the functions of BrainEx. 
from brainex.utils.gxe_utils import from_csv, from_db

In [6]:
# First, let's discuss the from_csv and from_db functions. 
# These functions (as you may recall from the saving + loading tutorial) are used to create a BrainEx Engine object. from_csv
# creates one based off of a datafile, and from_db loads a previously saved BrainEx Engine. Both from_csv and from_db have 
# parameters that allow you to define the resources given to your BrainEx Engine. 

# from_csv

data = './ItalyPower.csv' # The first parameter of from_csv is data. This is a path to the data file you wish to use. 

feature_num = 0 # Feature num is the number of columns before the timeseries data begins in your data file. Some data files 
# have data stored before the timeseries itself begins on each row. If your dataset has these columns then feature_num must
# be equal to the number of these columns. The dataset we're testing on doesn't have any so feature_num is 0.

use_spark = False # If this bool is set to True then your BrainEx Engine will use Spark. If it's set to False it won't. 

header = 0 # This is the number of header rows before the data in your dataset. Header rows usually name the different columns
# and most datasets either have one or none. Our dataset this time doesn't have any. 

driver_mem = 1 # This parameter defines the amount of memory (RAM) given to the main BrainEx process. In effect, 
# this is the number of gigabytes of RAM that BrainEx will run with. This number is limited by the amount of RAM available
# on your computer. For larger datasets I would suggest using more RAM, but our test dataset is small so 1 gb should be fine.

max_result_mem = 1 # This parameter defines the amount of memory (RAM) given to make the result. As with driver_mem, 
# larger datasets will require more RAM. If you are unsure how much to use I would suggest giving driver_mem and max_result_mem
# half of your memory each.

_rows_to_consider = None # By default BrainEx uses all rows of the inputted dataset, but if you would like to limit the number
# of rows that the system will search through then you can use this parameter. If _rows_to_consider is not included in the 
# from_csv invocation then the whole dataset will be searched through. If in doubt I would suggest not using this parameter. 

num_worker = 1 # num_worker sets the number of cores that BrainEx will use to preprocess the dataset. The larger the number of
# cores the faster your dataset will be processed, but you are limited by the number of cores in your computer. 

# Now, let's use the parameters we set above. 
brainexengine = from_csv(data, feature_num = feature_num, use_spark = use_spark, header = header, driver_mem = driver_mem, max_result_mem = max_result_mem, num_worker = num_worker)

msg: from_csv, feature num is 0, auto-generating uuid


ValueError: cannot insert uuid, already exists